# Введение в анализ данных (первый семестр)
## Домашнее задание №4 - Участие в конкурсе Quora Question Pairs

<b>Что вам нужно будет сделать</b><br>
До <b>23:59 7 мая</b> сделать хотя бы один submission в соревновании, а в промежуток <b>с 9 по 10 мая</b> выложить отчет (код в jupyter notebook, который генерирует ваше решение) в топик на kaggle, который я чуть позже создам (правила kaggle запрещают обмениваться кодом за пределами платформы).
Баллы за это задание будут начисляться следующим образом (только при наличии отчета):<br>
результат < 0.5 - 6 баллов<br>
результат < 0.4 - 8 баллов<br>
результат < 0.35 - 10 баллов<br>
остальные критерии будут определены постфактум (при оценивании будут учитываться ваше место в лидерборде и отчет)

<h3>Подготовка</h3>

Импорт всех необходимых пакетов:

In [1]:
import gc
import os
import time
from datetime import datetime
import cPickle
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
import string
from sklearn.metrics import hamming_loss
from sklearn.metrics.pairwise import paired_distances

In [3]:
def save_submission(submission, name='submission', path='./answers', with_dttm=False):
    suffix = '_' + datetime.now().strftime('%Y-%m-%d_%H-%M-%S') if with_dttm else ''
    filename = os.path.join(path, '%s%s.csv' % (name, suffix))
    submission.to_csv(filename, index=False)
    return filename

In [4]:
#models_and_test_matricies = {}
#with open('models_and_test_matricies_2.pkl', 'r') as fid:
    #models_and_test_matricies = cPickle.load(fid)
#models_and_test_matricies

{'BOW_words_not_equal': (LogisticRegression(C=0.1, class_weight={0: 1.32, 1: 0.46}, dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=10000,
            multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
            solver='sag', tol=0.0001, verbose=0, warm_start=False),
  <2345796x9281 sparse matrix of type '<type 'numpy.int32'>'
  	with 29083038 stored elements in Compressed Sparse Row format>),
 'TFIDF': (LogisticRegression(C=0.1, class_weight={0: 1.32, 1: 0.46}, dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=10000,
            multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
            solver='sag', tol=0.0001, verbose=0, warm_start=False),
  array([[ 0.66066394],
         [ 0.38021914],
         [ 0.31027794],
         ..., 
         [ 1.        ],
         [ 0.11289055],
         [ 0.38710213]]))}

<h3>Загрузка и предобработка тренировочных данных</h3>

In [4]:
train_df = pd.read_csv('data/train.csv')
train_df = train_df.dropna().reset_index(drop=True)
train_df.head(n=10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [5]:
#train_df = pd.read_csv('data/train.csv')
#train_df = train_df.dropna().reset_index(drop=True)

<h3>Извлечение признаков: Bag of words (chars)</h3>

In [6]:
BOW = CountVectorizer(max_df=0.999, min_df=100, analyzer='char', 
                      ngram_range=(1,3), max_features=None, binary=True, lowercase=True)
BOW.fit(pd.concat((train_df['question1'],train_df['question2'])).unique())

BOW_q1 = BOW.transform(train_df.question1)
BOW_q2 = BOW.transform(train_df.question2)

Введём следующий признак:<br>
0, если последовательность букв находится как в BOW первых вопросов, так и в BOW вторых вопросов, или отсутствует и там, и там<br>
-1, если последовательность букв находится только в одном BOW, и отсутствует в другом<br>

In [7]:
X_BOW_chars = -(BOW_q1 != BOW_q2).astype(int)
y = train_df.is_duplicate.values
X_BOW_chars.shape

(404288, 8345)

<h3>Обучение модели на тренировочных данных</h3>

Информация о весах классов взята [отсюда](https://www.kaggle.com/davidthaler/how-many-1-s-are-in-the-public-lb)

In [8]:
model_lr_bow_chars = LogisticRegression(C=0.1, solver='sag', class_weight={1: 0.46, 0: 1.32}, max_iter=10000).fit(X_BOW_chars, y)

<h3>Загрука и обработка тестовых данных</h3>

In [9]:
test_df = pd.read_csv('data/test.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345796 entries, 0 to 2345795
Data columns (total 3 columns):
test_id      int64
question1    object
question2    object
dtypes: int64(1), object(2)
memory usage: 53.7+ MB


In [10]:
test_df.loc[test_df['question1'].isnull(), ['question1','question2']] = ''
test_df.loc[test_df['question2'].isnull(), ['question1','question2']] = ''

In [11]:
#test_df = pd.read_csv('data/test.csv')
#test_df.loc[test_df['question1'].isnull(), ['question1','question2']] = ''
#test_df.loc[test_df['question2'].isnull(), ['question1','question2']] = ''

In [ ]:
gc.collect()
BOW_q1 = BOW.transform(test_df.loc[:,'question1'])
gc.collect()
BOW_q2 = BOW.transform(test_df.loc[:,'question2'])
gc.collect()

In [ ]:
X_BOW_chars = -(BOW_q1 != BOW_q2).astype(int)
X_BOW_chars.shape

(2345796, 8345)

<h3>Применение модели на тестовых данных</h3>

In [ ]:
predicted = model_lr_bow_chars.predict_proba(X_BOW_chars)

<h3>Сохранение предсказаний</h3>

In [ ]:
submission = pd.concat([test_df.test_id.astype(int), pd.DataFrame(predicted[:,1], 
                                        columns=['is_duplicate'], index=test_df.index)], axis=1)
print submission.shape
save_submission(submission, 'submission_BOW_chars', with_dttm=True)

(2345796, 2)


'./answers\\submission_BOW_chars_2017-05-10_16-39-21.csv'

In [ ]:
#models_and_test_matricies['BOW_chars_not_equal'] = (model_lr_bow_chars, X_BOW_chars)

In [18]:
#with open('models_and_test_matricies_2.pkl', 'wb') as fid:
    #cPickle.dump(models_and_test_matricies, fid)

<h3>Извлечение признаков: Bag of words (word) </h3>

In [51]:
del_punct_trans_table = dict((ord(char), None) for char in string.punctuation)

def nlp_preprocessor(sentence):
    return delete_punctuation_unicode(sentence.lower())

def nlp_tokenizer(sentence):
    tokens = word_tokenize(sentence)
    ll = WordNetLemmatizer()
    stemmer = SnowballStemmer(language='english', ignore_stopwords=True)
    tokens = [ll.lemmatize(item) for item in tokens]
    tokens = [stemmer.stem(item) for item in tokens]
    return tokens

def delete_punctuation_unicode(to_translate):
    return to_translate.translate(del_punct_trans_table)

In [52]:
BOW = CountVectorizer(max_df=0.999, min_df=20, preprocessor=nlp_preprocessor, tokenizer=nlp_tokenizer, analyzer='word',
                    max_features=None, binary=True)
BOW.fit(pd.concat((train_df['question1'],train_df['question2'])).unique())

BOW_q1 = BOW.transform(train_df.question1)
BOW_q2 = BOW.transform(train_df.question2)

In [53]:
y = train_df.is_duplicate.values
X_BOW_words_not_equal = -(BOW_q1 != BOW_q2).astype(int)
#X_ham_loss = paired_distances(BOW_q1, BOW_q2, metric=hamming_loss)
print X_BOW_words_not_equal.shape

(404288, 9281)


In [54]:
model_lr_BOW_words_not_equal = LogisticRegression(C=0.1, solver='sag', class_weight={1: 0.46, 0: 1.32}, 
                                                  max_iter=10000).fit(X_BOW_words_not_equal, y)

In [46]:
test_df = pd.read_csv('data/test.csv')
test_df.loc[test_df['question1'].isnull(), ['question1','question2']] = ''
test_df.loc[test_df['question2'].isnull(), ['question1','question2']] = ''

In [55]:
gc.collect()
BOW_q1 = BOW.transform(test_df.loc[:,'question1'])
gc.collect()
BOW_q2 = BOW.transform(test_df.loc[:,'question2'])
gc.collect()

0

In [56]:
X_BOW_words_not_equal = -(BOW_q1 != BOW_q2).astype(int)
X_BOW_words_not_equal.shape

(2345796, 9281)

In [57]:
del BOW_q1, BOW_q2

In [58]:
predicted = model_lr_BOW_words_not_equal.predict_proba(X_BOW_words_not_equal)

In [59]:
submission = pd.concat([test_df.test_id.astype(int), pd.DataFrame(predicted[:,1], 
                                        columns=['is_duplicate'], index=test_df.index)], axis=1)
print submission.shape
submission_BOW_words_not_equal = save_submission(submission, 'submission_BOW_words_not_equal', with_dttm=True)

(2345796, 2)


In [60]:
#models_and_test_matricies['BOW_words_not_equal'] = (model_lr_BOW_words_not_equal, X_BOW_words_not_equal)

In [62]:
#with open('models_and_test_matricies.pkl', 'wb') as fid:
    #cPickle.dump(models_and_test_matricies, fid)

<h3>Извлечение признаков: TF-IDF </h3>

In [28]:
from sklearn.metrics.pairwise import paired_cosine_distances

In [24]:
TFIDF = TfidfVectorizer(preprocessor=nlp_preprocessor, tokenizer=nlp_tokenizer, analyzer='word', max_df=0.999, min_df=20, 
                        norm='l2', use_idf=True, smooth_idf=True)
TFIDF.fit(pd.concat((train_df['question1'],train_df['question2'])).unique())

TFIDF_q1 = TFIDF.transform(train_df.question1)
TFIDF_q2 = TFIDF.transform(train_df.question2)

In [27]:
TFIDF_q1.shape

(404288, 9281)

In [37]:
y = train_df.is_duplicate.values
X_TFIDF = paired_cosine_distances(TFIDF_q1,TFIDF_q2).reshape(-1, 1)
X_TFIDF.shape

(404288L, 1L)

In [49]:
from sklearn.model_selection import cross_val_score, train_test_split

In [39]:
model_lr_tfidf = LogisticRegression(C=0.1, solver='sag', class_weight={1: 0.46, 0: 1.32}, max_iter=10000).fit(X_TFIDF, y)

In [40]:
test_df = pd.read_csv('data/test.csv')
test_df.loc[test_df['question1'].isnull(), ['question1','question2']] = ''
test_df.loc[test_df['question2'].isnull(), ['question1','question2']] = ''

In [41]:
gc.collect()
TFIDF_q1 = TFIDF.transform(test_df.loc[:,'question1'])
gc.collect()
TFIDF_q2 = TFIDF.transform(test_df.loc[:,'question2'])
gc.collect()

0

In [42]:
X_TFIDF = paired_cosine_distances(TFIDF_q1,TFIDF_q2).reshape(-1, 1)
del TFIDF_q1, TFIDF_q2

In [43]:
predicted = model_lr_tfidf.predict_proba(X_TFIDF)

In [44]:
submission = pd.concat([test_df.test_id.astype(int), pd.DataFrame(predicted[:,1], 
                                        columns=['is_duplicate'], index=test_df.index)], axis=1)
print submission.shape
submission_tfidf = save_submission(submission, 'submission_tfidf', with_dttm=True)

(2345796, 2)


In [45]:
#models_and_test_matricies['TFIDF'] = (model_lr_tfidf, X_TFIDF)

<h3>Использование нескольких моделей: усреднение</h3>

In [ ]:
#subm_1 = pd.read_csv('./answers/1/submission_BOW_chars_2017-05-08_19-53-25.csv')
#subm_2 = pd.read_csv('./answers/1/submission_BOW_words_2017-05-09_06-02-19.csv')
#subm_mean = pd.DataFrame(subm_1)
#subm_mean['is_duplicate'] = (subm_1['is_duplicate'] + subm_2['is_duplicate']) / 2
#save_submission(subm_mean, 'submission_BOW_mean', with_dttm=True)

submission_2017-05-07_21-33-18.csv 0.36967<br>
submission_BOW_words_2017-05-09_06-02-19.csv 0.36584<br>
submission_BOW_mean 0.35549